In [1]:
import os, sys
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW
reload(RTCW)

db_name = 'rtcw_db.db'
demos_folder = 'demos'
cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
root_path = 'C:\\Users\\Jelle\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

parsed 100 matches
parsed 200 matches
parsed 300 matches
parsed 400 matches
finished parsing all matches!


In [3]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [4]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
demo_df = db.get_table_as_df(table_name= 'demo')
player_df = db.get_table_as_df(table_name= 'player')
chatmessages_df = db.get_table_as_df(table_name= 'chatmessage')
chatmessages_df = RTCW.add_match_data(chatmessages_df, player_df, demos_dct, what_df = 'chatmessages_df')
chatmessages_df = RTCW.feature_extraction_chat(chatmessages_df)

obituary table fetched in 1.08 seconds
demo table fetched in 0.13 seconds
player table fetched in 0.16 seconds
chatmessage table fetched in 1.22 seconds


In [5]:
#docs_df = chatmessages_df.loc[chatmessages_df['DocsAll'] | chatmessages_df['TimelimitHit']]

In [6]:
chatmessages_df = chatmessages_df.loc[chatmessages_df['League'] == 'cb-nc3']

In [9]:
chatmessages_df[chatmessages_df['DynamiteAll'] == True].columns

Index([u'dwSeq', u'szMd5', u'szMessage', u'bPlayer', u'dwTime',
       u'szTimeString', u'szChatIdent', u'Date', u'Importance', u'League',
       u'TeamA', u'TeamB', u'demoName', u'matchName', u'Shoutcast', u'WTV',
       u'DocsAll', u'DynamiteAll', u'TimelimitHit', u'InMatch',
       u'SecondsLeftInRound', u'DocsEvents', u'DynamiteEvents'],
      dtype='object')

In [ ]:
#empty lists that we will populate and use to make final df
pd_md5 = []
pd_start_dyna_dwtime = []
pd_end_dyna_dwtime = []
pd_start_dyna_secsleft = []
pd_end_dyna_secsleft = []
pd_dyna_duration = []
pd_dyn_exploded = []
pd_demo_name = []
pd_match_name = []

#helper variables for verbose
counter = 0
total_demos = chatmessages_df.szMd5.nunique()

In [ ]:
demo = ''

In [30]:
chatmessages_df[(chatmessages_df['DynamiteEvents'] == True)].sort_values(['szMd5', 'dwTime'])

,dwSeq,szMd5,szMessage,bPlayer,dwTime,szTimeString,szChatIdent,Date,Importance,League,...,matchName,Shoutcast,WTV,DocsAll,DynamiteAll,TimelimitHit,InMatch,SecondsLeftInRound,DocsEvents,DynamiteEvents
268244,266092,006786ef8fcbfe3b4003814af4030cfd,Dynamite planted near The Communications Tower!,-1,19411494,6:00,cp,2002-10-24,groupstage,cb-ec6,...,rtcw_2002.10.24_cb-ec6_euph_vs_majesty_groupstage,False,0,0,1,0,1,360,-1,1
164456,164457,017271aeee41749de9d0cb57a3f9d565,Dynamite planted near the South Radar [02]!,-1,1144454600,13:10,cp,2003-01-26,None,wfl-wolfcup,...,rtcw_2003.01.26_wfl-wolfcup_marauders_vs_gsk,False,0,0,1,0,1,790,-1,1
164555,164556,017271aeee41749de9d0cb57a3f9d565,Dynamite planted near the North Radar [01]!,-1,1145258000,0:25,cp,2003-01-26,None,wfl-wolfcup,...,rtcw_2003.01.26_wfl-wolfcup_marauders_vs_gsk,False,0,0,1,0,1,25,-1,1
164642,164643,017271aeee41749de9d0cb57a3f9d565,Dynamite planted near the South Radar [02]!,-1,1145423550,14:30,cp,2003-01-26,None,wfl-wolfcup,...,rtcw_2003.01.26_wfl-wolfcup_marauders_vs_gsk,False,0,0,1,0,1,870,-1,1
164643,164644,017271aeee41749de9d0cb57a3f9d565,Dynamite planted near the North Radar [01]!,-1,1145461400,13:52,cp,2003-01-26,None,wfl-wolfcup,...,rtcw_2003.01.26_wfl-wolfcup_marauders_vs_gsk,False,0,0,1,0,1,832,-1,1
164650,164651,017271aeee41749de9d0cb57a3f9d565,Dynamite planted near the South Radar [02]!,-1,1145536100,12:37,cp,2003-01-26,None,wfl-wolfcup,...,rtcw_2003.01.26_wfl-wolfcup_marauders_vs_gsk,False,0,0,1,0,1,757,-1,1
164734,164735,017271aeee41749de9d0cb57a3f9d565,Dynamite planted near the South Radar [02]!,-1,1145740550,14:27,cp,2003-01-26,None,wfl-wolfcup,...,rtcw_2003.01.26_wfl-wolfcup_marauders_vs_gsk,False,0,0,1,0,1,867,-1,1
164737,164738,017271aeee41749de9d0cb57a3f9d565,Dynamite planted near the North Radar [01]!,-1,1145837250,12:50,cp,2003-01-26,None,wfl-wolfcup,...,rtcw_2003.01.26_wfl-wolfcup_marauders_vs_gsk,False,0,0,1,0,1,770,-1,1
164739,164740,017271aeee41749de9d0cb57a3f9d565,Dynamite planted near the North Radar [01]!,-1,1145879700,12:07,cp,2003-01-26,None,wfl-wolfcup,...,rtcw_2003.01.26_wfl-wolfcup_marauders_vs_gsk,False,0,0,1,0,1,727,-1,1
118459,118460,01bbd334c7651f060b651ca28eddf7a4,Dynamite planted near the Axis Submarine!,-1,1760700600,0:54,cp,2005-02-01,finals,cb-nc7,...,rtcw_2005.02.01_cb-nc7_bel_vs_ned_finals,False,0,0,1,0,1,54,-1,1


In [27]:
chatmessages_df[(chatmessages_df['DynamiteEvents'] == True) & (chatmessages_df['szMd5'] != '17d139cab4b1f9c9753832c66d83b784')]

,dwSeq,szMd5,szMessage,bPlayer,dwTime,szTimeString,szChatIdent,Date,Importance,League,...,matchName,Shoutcast,WTV,DocsAll,DynamiteAll,TimelimitHit,InMatch,SecondsLeftInRound,DocsEvents,DynamiteEvents
4794,4795,d6d63c0fd9edea6fecc0ce076f7760dc,Dynamite planted near The Communications Tower!,-1,55673720,5:32,cp,2003-08-07,None,liga-sw-s4,...,rtcw_2003.08.07_liga-sw-s4_knights-of-death_vs...,False,0,0,1,0,1,332,-1,1
5244,5245,6fe5698290962e4983fe133aea0ec9ae,Dynamite planted near The Communications Tower!,-1,56533080,4:32,cp,2003-08-07,None,liga-sw-s4,...,rtcw_2003.08.07_liga-sw-s4_knights-of-death_vs...,False,0,0,1,0,1,272,-1,1
5797,5798,97d2e29462726cd769821ab155c96c20,Dynamite planted near The Communications Tower!,-1,57533600,3:35,cp,2003-08-07,None,liga-sw-s4,...,rtcw_2003.08.07_liga-sw-s4_knights-of-death_vs...,False,0,0,1,0,1,215,-1,1
6521,6522,6edf369d3ae963e7055d98de0a1c0afc,Dynamite planted near The Communications Tower!,-1,58743960,3:35,cp,2003-08-07,None,liga-sw-s4,...,rtcw_2003.08.07_liga-sw-s4_knights-of-death_vs...,False,0,0,1,0,1,215,-1,1
6741,6742,0e617ede95893a53a56c704cb1b196fe,Dynamite planted near The Communications Tower!,-1,59194680,1:52,cp,2003-08-07,None,liga-sw-s4,...,rtcw_2003.08.07_liga-sw-s4_knights-of-death_vs...,False,0,0,1,0,1,112,-1,1
6869,6870,ddb5723dac19ab71dc1298025256e57d,Dynamite planted near The Communications Tower!,-1,4204100,7:14,cp,2002-11-20,round2,esports,...,rtcw_2002.11.20_esports_ocrana_vs_euphoria_round2,False,0,0,1,0,1,434,-1,1
7104,7105,35662fffb8929c6597ecd84f05248b06,Dynamite planted near The Communications Tower!,-1,5647700,5:23,cp,2002-11-20,round2,esports,...,rtcw_2002.11.20_esports_ocrana_vs_euphoria_round2,False,0,0,1,0,1,323,-1,1
7721,7722,f32b8c2d35fb5773968b21602513405f,Dynamite planted near the South Radar [02]!,-1,21060200,14:20,cp,2003-06-14,wb-round1,swertcwcup,...,rtcw_2003.06.14_swertcwcup_amish_vs_newproyect...,False,0,0,1,0,1,860,-1,1
7723,7724,f32b8c2d35fb5773968b21602513405f,Dynamite planted near the North Radar [01]!,-1,21127550,13:13,cp,2003-06-14,wb-round1,swertcwcup,...,rtcw_2003.06.14_swertcwcup_amish_vs_newproyect...,False,0,0,1,0,1,793,-1,1
7725,7726,f32b8c2d35fb5773968b21602513405f,Dynamite planted near the North Radar [01]!,-1,21188500,12:12,cp,2003-06-14,wb-round1,swertcwcup,...,rtcw_2003.06.14_swertcwcup_amish_vs_newproyect...,False,0,0,1,0,1,732,-1,1


In [ ]:
def get_docruns(chatmessages_df, min_docrun_length = None, max_timeleft = None, docs_succesful = None, min_docs_lost = None, verbose = True):
	
	#empty lists that we will populate and use to make final df
	pd_md5 = []
	pd_start_docrun_dwtime = []
	pd_end_docrun_dwtime = []
	pd_start_docrun_secsleft = []
	pd_end_docrun_secsleft = []
	pd_docrun_duration = []
	pd_times_docs_lost = []
	pd_won_round = []
	pd_demo_name = []
	pd_match_name = []
	
	#helper variables for verbose
	counter = 0
	total_demos = chatmessages_df.szMd5.nunique()
	
	for demo in chatmessages_df.szMd5.unique():
		#evt hier nog cutten op timelimit hit of docsevents zoals cell hierboven voor speed
		df_demo = chatmessages_df.loc[(chatmessages_df['szMd5'] == demo) & (chatmessages_df['InMatch'])]
		demo_name = df_demo.demoName.unique()
		match_name = df_demo.matchName.unique()
		counter += 1
		
		arr = df_demo.as_matrix(columns = ['DocsEvents', 'TimelimitHit', 'dwTime', 'SecondsLeftInRound'])
		taken_docs_bool = False
		docs_lost_times = 0
		
		for row in range(len(arr)):
			chat = arr[row, :]
			
			#taken docs
			if chat[0] == 1:
				if taken_docs_bool == False: 
					start_dwtime = chat[2]
					start_timeleft = chat[3]
					taken_docs_bool = True
				else:
					docs_lost_times += 1

			#transmitted docs
			if chat[0] == 2:
				end_dwtime = chat[2]
				end_timeleft = chat[3]
				taken_docs_bool = False
				docs_succes = True
				
				pd_md5.append(demo)
				pd_start_docrun_dwtime.append(start_dwtime)
				pd_end_docrun_dwtime.append(end_dwtime)
				pd_start_docrun_secsleft.append(start_timeleft)
				pd_end_docrun_secsleft.append(end_timeleft)
				pd_docrun_duration.append(start_timeleft - end_timeleft)
				pd_times_docs_lost.append(docs_lost_times)
				pd_won_round.append(docs_succes)
				pd_demo_name.append(demo_name[0])
				pd_match_name.append(match_name[0])
			
				docs_lost_times = 0
				
			#returned docs / timelimit hit
			if ((chat[0] == 0 or chat[1] == 1) and (taken_docs_bool == True)):
				end_dwtime = chat[2]
				end_timeleft = chat[3]
				taken_docs_bool = False
				docs_succes = False
				
				pd_md5.append(demo)
				pd_start_docrun_dwtime.append(start_dwtime)
				pd_end_docrun_dwtime.append(end_dwtime)
				pd_start_docrun_secsleft.append(start_timeleft)
				pd_end_docrun_secsleft.append(end_timeleft)
				pd_docrun_duration.append(start_timeleft - end_timeleft)
				pd_times_docs_lost.append(docs_lost_times)
				pd_won_round.append(docs_succes)
				pd_demo_name.append(demo_name[0])
				pd_match_name.append(match_name[0])
				
				docs_lost_times = 0
				
			#if demo stopped before end of docrun
			
			
		#verbose shizzle
		if counter % 100 == 0:
			if verbose:
				print 'scanned ' + str(counter) + ' demos of ' + str(total_demos) + ' demos in total' 
	
	print 'all done!'
	
	
	#make final dataframe where 1 row is a spree with all the necessary info
	df_docs = pd.DataFrame(
	{'md5': pd_md5,
	 'start': pd_start_docrun_dwtime,
	 'end': pd_end_docrun_dwtime,
	 'start_secsleft': pd_start_docrun_secsleft,
	 'end_secsleft': pd_end_docrun_secsleft,
	 'duration': pd_docrun_duration,
	 'times_lost_docs': pd_times_docs_lost,
	 'won_round': pd_won_round,
	 'demo': pd_demo_name,
	 'match': pd_match_name,
	})
	
	if min_docrun_length != None:
		df_docs = df_docs.loc[df_docs['duration'] >= min_docrun_length]
		
	if max_timeleft != None:
		df_docs = df_docs.loc[df_docs['end_secsleft'] <= max_timeleft]
		
	if docs_succesful != None:
		df_docs = df_docs.loc[df_docs['won_round'] == docs_succesful]
		
	if min_docs_lost != None:
		df_docs = df_docs.loc[df_docs['times_lost_docs'] >= min_docs_lost]
		
	return df_docs     


In [7]:
def get_docruns(chatmessages_df, min_docrun_length = None, max_timeleft = None, docs_succesful = None, min_docs_lost = None, verbose = True):
    
    #empty lists that we will populate and use to make final df
    pd_md5 = []
    pd_start_docrun_dwtime = []
    pd_end_docrun_dwtime = []
    pd_start_docrun_secsleft = []
    pd_end_docrun_secsleft = []
    pd_docrun_duration = []
    pd_times_docs_lost = []
    pd_won_round = []
    pd_demo_name = []
    pd_match_name = []
    
    #helper variables for verbose
    counter = 0
    total_demos = obituary_df.szMd5.nunique()
    
    for demo in chatmessages_df.szMd5.unique():
        #evt hier nog cutten op timelimit hit of docsevents zoals cell hierboven voor speed
        df_demo = chatmessages_df.loc[(chatmessages_df['szMd5'] == demo) & (chatmessages_df['InMatch'])]
        demo_name = df_demo.demoName.unique()
        match_name = df_demo.matchName.unique()
        counter += 1
        
        arr = df_demo.as_matrix(columns = ['DocsEvents', 'TimelimitHit', 'dwTime', 'SecondsLeftInRound'])
        taken_docs_bool = False
        docs_lost_times = 0
        
        for row in range(len(arr)):
            chat = arr[row, :]
            
            #taken docs
            if chat[0] == 1:
                if taken_docs_bool == False: 
                    start_dwtime = chat[2]
                    start_timeleft = chat[3]
                    taken_docs_bool = True
                else:
                    docs_lost_times += 1

            #transmitted docs
            if chat[0] == 2:
                end_dwtime = chat[2]
                end_timeleft = chat[3]
                taken_docs_bool = False
                docs_succes = True
                
                pd_md5.append(demo)
                pd_start_docrun_dwtime.append(start_dwtime)
                pd_end_docrun_dwtime.append(end_dwtime)
                pd_start_docrun_secsleft.append(start_timeleft)
                pd_end_docrun_secsleft.append(end_timeleft)
                pd_docrun_duration.append(start_timeleft - end_timeleft)
                pd_times_docs_lost.append(docs_lost_times)
                pd_won_round.append(docs_succes)
                pd_demo_name.append(demo_name[0])
                pd_match_name.append(match_name[0])
            
                docs_lost_times = 0
                
            #returned docs / timelimit hit
            if ((chat[0] == 0 or chat[1] == 1) and (taken_docs_bool == True)):
                end_dwtime = chat[2]
                end_timeleft = chat[3]
                taken_docs_bool = False
                docs_succes = False
                
                pd_md5.append(demo)
                pd_start_docrun_dwtime.append(start_dwtime)
                pd_end_docrun_dwtime.append(end_dwtime)
                pd_start_docrun_secsleft.append(start_timeleft)
                pd_end_docrun_secsleft.append(end_timeleft)
                pd_docrun_duration.append(start_timeleft - end_timeleft)
                pd_times_docs_lost.append(docs_lost_times)
                pd_won_round.append(docs_succes)
                pd_demo_name.append(demo_name[0])
                pd_match_name.append(match_name[0])
                
                docs_lost_times = 0
                
            #if demo stopped before end of docrun
            
            
        #verbose shizzle
        if counter % 100 == 0:
            if verbose:
                print 'scanned ' + str(counter) + ' demos of ' + str(total_demos) + ' demos in total' 
    
    print 'all done!'
    
    
    #make final dataframe where 1 row is a spree with all the necessary info
    df_docs = pd.DataFrame(
    {'md5': pd_md5,
     'start_dwtime': pd_start_docrun_dwtime,
     'end_dwtime': pd_end_docrun_dwtime,
     'start_secsleft': pd_start_docrun_secsleft,
     'end_secsleft': pd_end_docrun_secsleft,
     'duration': pd_docrun_duration,
     'times_lost_docs': pd_times_docs_lost,
     'won_round': pd_won_round,
     'demo': pd_demo_name,
     'match': pd_match_name,
    })
    
    
    
    if min_docrun_length != None:
        df_docs = df_docs.loc[df_docs['duration'] >= min_docrun_length]
        
    if max_timeleft != None:
        df_docs = df_docs.loc[df_docs['end_secsleft'] <= max_timeleft]
        
    if docs_succesful != None:
        df_docs = df_docs.loc[df_docs['won_round'] == docs_succesful]
        
    if min_docs_lost != None:
        df_docs = df_docs.loc[df_docs['times_lost_docs'] >= min_docs_lost]
        
    return df_docs    

In [17]:
reload(RTCW)
res = RTCW.get_docruns(chatmessages_df, 
                       min_docrun_length = 10, 
                       max_timeleft = 999, 
                       docs_succesful = True, 
                       min_docs_lost = 2, 
                       verbose = True)
res[['start', 'end', 'start_secsleft', 'end_secsleft', 'duration', 'times_lost_docs', 'won_round', 'md5']]

all done!


,start,end,start_secsleft,end_secsleft,duration,times_lost_docs,won_round,md5
44,9112750,9184000,344,273,71,2,True,37e1c83294ad714e2a7d59d2b5bbee48
46,603061814,603090425,35,7,28,2,True,bcf458553680b8d93a166f526ce1367d
80,20337300,20438700,310,208,102,2,True,3eba777ce461666c493c7b47f8a8daf5


In [18]:
res

,demo,duration,end,end_secsleft,match,md5,start,start_secsleft,times_lost_docs,won_round
44,222118-MVD-mp_village.dm_60,71,9184000,273,rtcw_2003.01.09_cb-nc3_swe_vs_nor_quarter-finals,37e1c83294ad714e2a7d59d2b5bbee48,9112750,344,2,True
46,CBNC_Winter2002_GER_vs_FRA_beach1.dm_60,28,603090425,7,rtcw_2002.12.16_cb-nc3_ger_vs_fra_groupstage,bcf458553680b8d93a166f526ce1367d,603061814,35,2,True
80,SWE-DEN.APE.dm_60,102,20438700,208,rtcw_2002.12.16_cb-nc3_swe_vs_den_groupstage,3eba777ce461666c493c7b47f8a8daf5,20337300,310,2,True


In [19]:
reload(RTCW)
RTCW.cut_demos(root_path, demos_dct, res, demo_type = 'docs', offset_start = 8, offset_end = 8, transform_to_dm_60 = True, demo_folder_name= 'demos')

In [20]:
reload(RTCW)
RTCW.generate_capture_list(res, demo_type='docs', transform_to_dm_60 = True)

In [ ]:
spree = res.iloc[0]
spree['demo'][:-6] + '_' + str(spree['duration']) + '_' + str(spree['start_secsleft']) + '_' + str(spree['end_secsleft']) + '_' + str(spree['won_round']) + '_' + str(spree['start_dwtime']) + spree['demo'][-6:] 